## 国内huggingface端点

In [1]:
HF_ENDPOINT='https://hf-mirror.com'

In [2]:
def download(repo_id: str, repo_type: str):
    from huggingface_hub import snapshot_download
    snapshot_download(repo_id=repo_id,
                      repo_type=repo_type,
                      resume_download=False,
                      force_download=True,
                      endpoint=HF_ENDPOINT,
                      local_dir=f'./{repo_id}')

## 1. 创建embedding模型

In [3]:
!pip install sentence_transformers

In [4]:
download('BAAI/bge-large-zh-v1.5', 'model')

E:\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 12 files: 100%|███████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 21.47it/s]


In [5]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm_notebook as tqdm
model_name = 'BAAI/bge-large-zh-v1.5'
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},# 根据你的需要去选择设备
    encode_kwargs={'normalize_embeddings': True} ,# set True to compute cosine similarity
    query_instruction='',
    cache_folder=f'./{model_name}'
)

E:\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## 2. faiss

In [6]:
!pip install faiss-cpu
# !conda install -c pytorch faiss-gpu

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

In [8]:
# 实例化文档加载器
loader = TextLoader("./data/西游记.txt", 'utf-8')
# 加载文档
documents = loader.load()
# 实例化文本分割器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# 分割文本
docs = text_splitter.split_documents(documents)

In [9]:
docs[:10]

[Document(page_content='《》目录 第一回\u3000灵根育孕源流出\u3000心性修持大道生\n \n    诗曰：\n \n    混沌未分天地乱，茫茫渺渺无人见。\n \n    自从盘古破鸿蒙，开辟从兹清浊辨。\n \n    覆载群生仰至仁，发明万物皆成善。\n \n    欲知造化会元功，须看西游释厄传。', metadata={'source': './data/西游记.txt'}),
 Document(page_content='盖闻天地之数，有十二万九千六百岁为一元。将一元分为十二会，乃子、丑、寅、卯、辰、巳、午、未、申、酉、戌、亥之十二支也。每会该一万八百岁。且就一日而论：子时得阳气，而丑则鸡鸣；寅不通光，而卯则日出；辰时食后，而巳则挨排；日午天中，而未则西蹉；申时晡而日落酉；戌黄昏而人定亥。譬于大数，若到戌会之终，则天地昏蒙而万物否矣。再去五千四百岁，交亥会之初，则当黑暗，而两间人物俱无矣，故曰混沌。又五千四百岁，亥会将终，贞下起元，近子之会，而复逐渐开明。邵康节曰：“冬至子之半，天心无改移。一阳初动处，万物未生时。”到此，天始有根。再五千四百岁，正当子会，轻清上腾，有日，有月，有星，有辰。日、月、星、辰，谓之四象。故曰，天开于子。又经五千四百岁，子会将终，近丑之会，而逐渐坚实。易曰：“大哉乾元！至哉坤元！万物资生，乃顺承天。”至此，地始凝结。再五千四百岁，正当丑会，重浊下凝，有水，有火，有山，有石，有土。水、火、山、石、土谓之五形。故曰，地辟于丑。又经五千四百岁，丑会终而寅会之初，发生万物。历曰：“天气下降，地气上升；天地交合，群物皆生。”至此，天清地爽，阴阳交合。再五千四百岁，正当寅会，生人，生兽，', metadata={'source': './data/西游记.txt'}),
 Document(page_content='生禽，正谓天地人，三才定位。故曰，人生于寅。', metadata={'source': './data/西游记.txt'}),
 Document(page_content='感盘古开辟，三皇治世，五帝定伦，世界之间，遂分为四大部洲：曰东胜神洲，曰西牛贺洲，曰南赡部洲，曰北俱芦洲。这部书单表东胜神洲。海外有一国土，名曰傲来国。国近大海，海中有一座山，唤为花果山。此山乃十洲之祖脉，三岛之来龙

In [10]:
# FAISS 向量数据库，使用 docs 的向量作为初始化存储
db = FAISS.from_documents(docs[:10], embedding_model)

In [11]:
# 构造提问 Query
query = "又经五千四百岁，丑会终而寅会之初，发生万物"
# 在 Faiss 中进行相似度搜索，找出与 query 最相似结果
docs = db.similarity_search(query)
# 输出 Faiss 中最相似结果
print(docs[0].page_content)

盖闻天地之数，有十二万九千六百岁为一元。将一元分为十二会，乃子、丑、寅、卯、辰、巳、午、未、申、酉、戌、亥之十二支也。每会该一万八百岁。且就一日而论：子时得阳气，而丑则鸡鸣；寅不通光，而卯则日出；辰时食后，而巳则挨排；日午天中，而未则西蹉；申时晡而日落酉；戌黄昏而人定亥。譬于大数，若到戌会之终，则天地昏蒙而万物否矣。再去五千四百岁，交亥会之初，则当黑暗，而两间人物俱无矣，故曰混沌。又五千四百岁，亥会将终，贞下起元，近子之会，而复逐渐开明。邵康节曰：“冬至子之半，天心无改移。一阳初动处，万物未生时。”到此，天始有根。再五千四百岁，正当子会，轻清上腾，有日，有月，有星，有辰。日、月、星、辰，谓之四象。故曰，天开于子。又经五千四百岁，子会将终，近丑之会，而逐渐坚实。易曰：“大哉乾元！至哉坤元！万物资生，乃顺承天。”至此，地始凝结。再五千四百岁，正当丑会，重浊下凝，有水，有火，有山，有石，有土。水、火、山、石、土谓之五形。故曰，地辟于丑。又经五千四百岁，丑会终而寅会之初，发生万物。历曰：“天气下降，地气上升；天地交合，群物皆生。”至此，天清地爽，阴阳交合。再五千四百岁，正当寅会，生人，生兽，


In [12]:
# 持久化
db.save_local("faiss_index")

In [13]:
# 加载数据库
new_db = FAISS.load_local("faiss_index", embedding_model)

In [14]:
docs = new_db.similarity_search(query)
print(docs[0].page_content)

盖闻天地之数，有十二万九千六百岁为一元。将一元分为十二会，乃子、丑、寅、卯、辰、巳、午、未、申、酉、戌、亥之十二支也。每会该一万八百岁。且就一日而论：子时得阳气，而丑则鸡鸣；寅不通光，而卯则日出；辰时食后，而巳则挨排；日午天中，而未则西蹉；申时晡而日落酉；戌黄昏而人定亥。譬于大数，若到戌会之终，则天地昏蒙而万物否矣。再去五千四百岁，交亥会之初，则当黑暗，而两间人物俱无矣，故曰混沌。又五千四百岁，亥会将终，贞下起元，近子之会，而复逐渐开明。邵康节曰：“冬至子之半，天心无改移。一阳初动处，万物未生时。”到此，天始有根。再五千四百岁，正当子会，轻清上腾，有日，有月，有星，有辰。日、月、星、辰，谓之四象。故曰，天开于子。又经五千四百岁，子会将终，近丑之会，而逐渐坚实。易曰：“大哉乾元！至哉坤元！万物资生，乃顺承天。”至此，地始凝结。再五千四百岁，正当丑会，重浊下凝，有水，有火，有山，有石，有土。水、火、山、石、土谓之五形。故曰，地辟于丑。又经五千四百岁，丑会终而寅会之初，发生万物。历曰：“天气下降，地气上升；天地交合，群物皆生。”至此，天清地爽，阴阳交合。再五千四百岁，正当寅会，生人，生兽，


In [15]:
new_db.similarity_search_with_score('生禽，正谓天地人，三才定位。故曰，人生于寅。')

[(Document(page_content='生禽，正谓天地人，三才定位。故曰，人生于寅。', metadata={'source': './data/西游记.txt'}),
  0.18734923),
 (Document(page_content='盖闻天地之数，有十二万九千六百岁为一元。将一元分为十二会，乃子、丑、寅、卯、辰、巳、午、未、申、酉、戌、亥之十二支也。每会该一万八百岁。且就一日而论：子时得阳气，而丑则鸡鸣；寅不通光，而卯则日出；辰时食后，而巳则挨排；日午天中，而未则西蹉；申时晡而日落酉；戌黄昏而人定亥。譬于大数，若到戌会之终，则天地昏蒙而万物否矣。再去五千四百岁，交亥会之初，则当黑暗，而两间人物俱无矣，故曰混沌。又五千四百岁，亥会将终，贞下起元，近子之会，而复逐渐开明。邵康节曰：“冬至子之半，天心无改移。一阳初动处，万物未生时。”到此，天始有根。再五千四百岁，正当子会，轻清上腾，有日，有月，有星，有辰。日、月、星、辰，谓之四象。故曰，天开于子。又经五千四百岁，子会将终，近丑之会，而逐渐坚实。易曰：“大哉乾元！至哉坤元！万物资生，乃顺承天。”至此，地始凝结。再五千四百岁，正当丑会，重浊下凝，有水，有火，有山，有石，有土。水、火、山、石、土谓之五形。故曰，地辟于丑。又经五千四百岁，丑会终而寅会之初，发生万物。历曰：“天气下降，地气上升；天地交合，群物皆生。”至此，天清地爽，阴阳交合。再五千四百岁，正当寅会，生人，生兽，', metadata={'source': './data/西游记.txt'}),
  0.9508221),
 (Document(page_content='猴，在那里拜四方，眼运金光，射冲斗府。如今服饵水食，金光将潜息矣。”玉帝垂赐恩慈曰：“下方之物，乃天地精华所生，不足为异。”', metadata={'source': './data/西游记.txt'}),
  1.0720758),
 (Document(page_content='感盘古开辟，三皇治世，五帝定伦，世界之间，遂分为四大部洲：曰东胜神洲，曰西牛贺洲，曰南赡部洲，曰北俱芦洲。这部书单表东胜神洲。海外有一国土，名曰傲来国。国近大海，海中有一座山，唤为花果山。此山乃十洲之祖脉，三岛之来龙，自开清浊而立，鸿蒙判后而成。真个好山！有词赋为证。赋曰：', metad

## 3. 智谱

In [16]:
!pip install zhipuai

In [17]:
from zhipuai import ZhipuAI
import os

ZHIPU_AI_KEY = os.environ.get('ZHIPU_AI_KEY')
client = ZhipuAI(api_key=ZHIPU_AI_KEY) # 填写您自己的APIKey
tokens_counter = {}

In [20]:
def ask_glm(system_prompt:str, prompt:str, model_name:str = 'glm-3-turbo'):
    response = client.chat.completions.create(
        model= model_name,  # 填写需要调用的模型名称
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,
    )
    
    for k, v in json.loads(response.usage.model_dump_json()).items():
        if k not in tokens_counter:
            tokens_counter[k] = 0
        tokens_counter[k] += v
        
    return response.choices[0].message.content

提取问题和答案

In [28]:
import json
def extract_qa(text, model_name:str = 'glm-3-turbo'):
    system_prompt = """你是一个专业的知识问答助手，你的任务是从用户输入的文本中找到问题和答案，\
    最后以json格式输出，输出格式为{"query": #问题, "answer": #答案}，不要包含其他输出和标注"""
    qa_json = ask_glm(system_prompt=system_prompt, prompt=text, model_name=model_name)
    
    start = qa_json.find("{")
    end = qa_json.find("}")
    
    try:
        return json.loads(qa_json[start: end + 1])
    
    except Exception as e:
        print(e, qa_json)
        return {}

In [27]:
extract_qa('弼马温是什么工作?养马')

{'query': '弼马温是什么工作?', 'answer': '养马'}

## 4. moonshot